In [1]:
import histogram_manager as hm
import ROOT
import utils
import numpy as np

%jsroot on

Welcome to JupyROOT 6.30/04


In [2]:
hists =  hm.HistogramManager("../output/BR_Bo.root")
hists

bin_num = 50

### Loading the 477 keV gated spectrum

In [3]:
hEn_gate477_all = hists.get("hEn_gate477")
# hEn_gate477_all.Rebin(4)
# hEn_gate477_all.Sumw2()

c = ROOT.TCanvas()
hEn_gate477_all.Draw("")
# hEn_gate477_all.GetXaxis().SetRangeUser(1,200)
c.Draw()

hEn_gate477_all_content = hEn_gate477_all.GetBinContent(bin_num)
hEn_gate477_all_error = hEn_gate477_all.GetBinError(bin_num)
hEn_gate477_all_expected =  np.sqrt(hEn_gate477_all_content)
print(f"Bin {bin_num}: content = {hEn_gate477_all_content}, error = {hEn_gate477_all_error}")
print(f"Expected error: {hEn_gate477_all_expected}")

Bin 50: content = 69447.0, error = 263.52798712850216
Expected error: 263.52798712850216


### Loading the 10B (n,gamma) cross section
Made the error 1% as a placeholder

In [4]:
bin_edges_array = hEn_gate477_all.GetXaxis().GetXbins()
xbins_En = np.array([bin_edges_array[i] for i in range(bin_edges_array.GetSize())])



hEn_sig_tot_10B = utils.get_cross_section("../data/nndc_data.txt", xbins_En)

for i in range(1, len(xbins_En)):
        # hEn_sig_tot_10B.SetBinContent(i, c[i])
        hEn_sig_tot_10B.SetBinError(i,0.4*hEn_sig_tot_10B.GetBinContent(i))
        # hEn_sig_tot_10B.SetBinError(i,0)
        
c = ROOT.TCanvas()
hEn_sig_tot_10B.Draw()
c.Draw()


hEn_sig_tot_10B_content = hEn_sig_tot_10B.GetBinContent(bin_num)
hEn_sig_tot_10B_error = hEn_sig_tot_10B.GetBinError(bin_num)
hEn_sig_tot_10B_expected =  0.4*hEn_sig_tot_10B_content
print(f"Bin {bin_num}: content = {hEn_sig_tot_10B_content}, error = {hEn_sig_tot_10B_error}")
print(f"Expected error: {hEn_sig_tot_10B_expected}")

Bin 50: content = 505.6836568280389, error = 202.27346273121557
Expected error: 202.27346273121557


In [5]:
# n = 1.41E23 # number density of 10Bo[atom/cm^3]
# d = 0.05 # target thickness [cm]
# f = 1E-24


# # hEn_divisor = hEn_gate477_all.Clone()
# hEn_divisor = hEn_sig_tot_10B.Clone()

# hEn_divisor.SetTitle("hEn_divisor")
# hEn_divisor.SetName("hEn_divisor")


# # for bin in range(1, hEn_divisor.GetNbinsX() + 1):  
# #     original_value = hEn_divisor.GetBinContent(bin)
# #     hEn_divisor.SetBinContent(bin, (1 - ROOT.TMath.Exp(-1 * n * d * f * original_value)))


# c = ROOT.TCanvas()
# # hEn_divisor.GetXaxis().SetRangeUser(1,200)
# hEn_divisor.Draw()
# c.Draw()

### Dividing the En spectrum by the cross section 

In [6]:
hEn_beam_intensity = hEn_gate477_all.Clone()

# hEn_gate477_all.Sumw2()
# hEn_divisor.Sumw2()

hEn_beam_intensity = hEn_gate477_all/hEn_sig_tot_10B

hEn_beam_intensity.Draw()
# hEn_beam_intensity.GetXaxis().SetRangeUser(1,200)
hEn_beam_intensity.SetName("hEn_beam_intensity")
hEn_beam_intensity.SetTitle("hEn_beam_intensity")
hEn_beam_intensity.GetYaxis().SetTitle("Intensity [A.U.]")

c = ROOT.TCanvas()
hEn_beam_intensity.Draw()
c.Draw()

hEn_beam_intensity_content = hEn_beam_intensity.GetBinContent(bin_num)
hEn_beam_intensity_error = hEn_beam_intensity.GetBinError(bin_num)
hEn_beam_intensity_expected =  hEn_beam_intensity_content * np.sqrt((hEn_gate477_all_error/hEn_gate477_all_content)**2+(hEn_sig_tot_10B_error/hEn_sig_tot_10B_content)**2)
print(f"Bin {bin_num}: content = {hEn_beam_intensity_content}, error = {hEn_beam_intensity_error}")
print(f"Expected error: {hEn_beam_intensity_expected}")

Bin 50: content = 137.33289391951996, error = 54.93562941331935
Expected error: 54.93562941331936


In [7]:
x_point = 4.5
bin_index = hEn_beam_intensity.FindBin(x_point)
entry = hEn_beam_intensity.GetBinContent(bin_index)

hEn_beam_intensity.Scale(1/entry)

hEn_beam_intensity_scaled_content = hEn_beam_intensity.GetBinContent(bin_num)
hEn_beam_intensity_scaled_error = hEn_beam_intensity.GetBinError(bin_num)
hEn_beam_intensity_scaled_expected = hEn_beam_intensity_expected * 1/entry
print(f"Bin {bin_num}: content = {hEn_beam_intensity_scaled_content}, error = {hEn_beam_intensity_scaled_error}")
print(f"Expected error: {hEn_beam_intensity_scaled_expected}")

Bin 50: content = 2.1049283391493057, error = 0.8420092221231787
Expected error: 0.842009222123179


In [8]:
# beam_hist_dict = {"hEn_gate477_all":hEn_gate477_all,
#                   "hEn_sig_tot_10B":hEn_sig_tot_10B,
#                   "hEn_divisor":hEn_divisor,
#                   "hEn_beam_intensity":hEn_beam_intensity}

beam_hist_dict = {"hEn_gate477_all":hEn_gate477_all,
                  "hEn_sig_tot_10B":hEn_sig_tot_10B,
                  "hEn_beam_intensity":hEn_beam_intensity}

hists_beam = hm.HistogramManager(beam_hist_dict)
hists_beam.write("../output/BR_beam.root")

In [9]:
# hists_cd = hm.HistogramManager("../output/2023dec/BR_Cd.root")
# hEn_all = hists_cd.get('hEn')

In [10]:
# hEn_all = hists_cd.get('hEn')
# # hEn_all.Rebin(4)


# c = ROOT.TCanvas()
# hEn_all.Draw()
# c.Draw()

# hEn_all_content = hEn_all.GetBinContent(bin_num)
# hEn_all_error = hEn_all.GetBinError(bin_num)
# hEn_all_expected =  np.sqrt(hEn_all_content)
# print(f"Bin {bin_num}: content = {hEn_all_content}, error = {hEn_all_error}")
# print(f"Expected error: {hEn_all_expected}")

In [11]:
# hEn_all_beam = hEn_all.Clone()


# hEn_all_beam = hEn_all_beam/hEn_beam_intensity

# c = ROOT.TCanvas()
# hEn_all.Draw("e")
# hEn_all_beam.Draw("same")
# hEn_all_beam.SetLineColor(ROOT.kRed)
# c.Draw()


# hEn_all_beam_content = hEn_all_beam.GetBinContent(bin_num)
# hEn_all_beam_error = hEn_all_beam.GetBinError(bin_num)
# hEn_all_beam_expected =  hEn_all_beam_content * np.sqrt((hEn_beam_intensity_error/hEn_beam_intensity_content)**2+(hEn_all_error/hEn_all_content)**2)
# print(f"Bin {bin_num}: content = {hEn_all_beam_content}, error = {hEn_all_beam_error}")
# print(f"Expected error: {hEn_all_beam_expected}")